<a href="https://colab.research.google.com/github/yeesem/Deep_Learning/blob/main/Build_Custom_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import backend as K

# Prepare the Dataset

In [64]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Build the Model

In [65]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = (28,28)),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Lambda(lambda x:tf.abs(x)),
    tf.keras.layers.Dense(10,activation = 'softmax')
])

In [66]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)
model.fit(
    epochs = 5,
    x = x_train,
    y = y_train
)
model.evaluate(
    x = x_test,
    y = y_test
)

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2178 - accuracy: 0.9381
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0885 - accuracy: 0.9738
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0626 - accuracy: 0.9804
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0467 - accuracy: 0.9857
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0850 - accuracy: 0.9746


[0.08497411757707596, 0.9746000170707703]

In [67]:
def my_relu(x):
  return K.maximum(-0.1,x)

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = (28,28)),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Lambda(my_relu),
    tf.keras.layers.Dense(10,activation = 'softmax')
])

model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

model.fit(
    x_train,
    y_train,
    epochs = 5
)

model.evaluate(x_test,y_test)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2530 - accuracy: 0.9272
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1118 - accuracy: 0.9666
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0767 - accuracy: 0.9779
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0566 - accuracy: 0.9830
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0725 - accuracy: 0.9778


[0.07246161252260208, 0.9778000116348267]

# Building a custom layer

In [105]:
# inherit from this base class
from tensorflow.keras.layers import Layer

class SimpleDense(Layer):

    def __init__(self, units=32):
        '''Initializes the instance attributes'''
        super(SimpleDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        '''Create the state of the layer (weights)'''
        # initialize the weights
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="kernel",
            initial_value=w_init(shape=(input_shape[-1], self.units),
                                 dtype='float32'),
            trainable=True)

        # initialize the biases
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
            initial_value=b_init(shape=(self.units,), dtype='float32'),
            trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

# Activation in Custom Dense Layer

In [89]:
class SimpleDense(Layer):

    # add an activation parameter
    def __init__(self, units=32, activation=None):
        super(SimpleDense, self).__init__()
        self.units = units

        # define the activation to get from the built-in activation layers in Keras
        self.activation = tf.keras.activations.get(activation)


    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="kernel",
            initial_value=w_init(shape=(input_shape[-1], self.units),
                                 dtype='float32'),
            trainable=True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
            initial_value=b_init(shape=(self.units,), dtype='float32'),
            trainable=True)
        super().build(input_shape)


    def call(self, inputs):

        # pass the computation to the activation layer
        return self.activation(tf.matmul(inputs, self.w) + self.b)

In [92]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    SimpleDense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3016 - accuracy: 0.9118
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1445 - accuracy: 0.9568
Epoch 3/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.1066 - accuracy: 0.9683
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0880 - accuracy: 0.9731
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0753 - accuracy: 0.9770


[0.07525032758712769, 0.9769999980926514]